1. Select the last 4 month range
2. Get all the speech urls from the returned result
3. Fetch each page and extract speech body
4. Analyze each speech for H and D using dictionaries.

In [152]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import os

In [153]:
FED_BASE_URL = "https://www.federalreserve.gov"
FED_SPEECHES_URL = "https://www.federalreserve.gov/newsevents/speeches.htm"

In [154]:
print("Instanciating Chrome web driver")

HOME_DIR = "/Users/dennisfenchenko/"

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

Instanciating Chrome web driver


In [155]:
START_DATE = "06/20/2022"
END_DATE = "10/20/2022"

def get_fed_speeches(start_date, end_date, pagination=1):
    START_DATE_XPATH = "//*[@id=\"content\"]/div[2]/div/div[1]/form/div[2]/div/div[1]/input"
    END_DATE_XPATH = "//*[@id=\"content\"]/div[2]/div/div[1]/form/div[2]/div/div[2]/input"
    SUBMIT_BUTTON_XPATH = "//*[@id=\"content\"]/div[2]/div/div[1]/form/div[5]/a"
    driver.get(FED_SPEECHES_URL)
    start_date_input_box = driver.find_element("xpath", START_DATE_XPATH)
    end_date_input_box = driver.find_element("xpath", END_DATE_XPATH)
    start_date_input_box.clear()
    start_date_input_box.send_keys(start_date)
    end_date_input_box.clear()
    end_date_input_box.send_keys(end_date)
    submit_button = driver.find_element("xpath", SUBMIT_BUTTON_XPATH)
    submit_button.click()
    return driver.page_source

In [156]:
doc_body = get_fed_speeches(START_DATE, END_DATE)

In [157]:
# def get_fed_speeches_transcripts(all_speeches):
#     SPEECH_TAG_NAME = 'em'
#     soup = BeautifulSoup(all_speeches, 'html.parser')
#     fed_speech_transcript_elem = soup.find_all(SPEECH_TAG_NAME)
#     print(fed_speech_transcript_elem[0])
#     print(type(fed_speech_transcript_elem[0]))
#     fed_speech_transcripts = [elem.attrs['href'] for elem in
#                               fed_speech_transcript_elem]
#     return fed_speech_transcripts

def get_fed_speech_links(all_speeches):
    soup = BeautifulSoup(all_speeches, 'html.parser')
    fed_page_links = soup.find_all("a", href=True)
    # print(fed_page_links)
    fed_speech_transcript_links = [link['href'] for \
            link in fed_page_links if "/speech/" in link['href']]
    return fed_speech_transcript_links

In [158]:
import re

def get_fed_speech_transcript_links(fed_speech_links):
    DATE_LENGTH = 8
    fed_speech_transcript_links = []
    for link in fed_speech_links:
        date = re.findall('[0-9]+', link)
        if len(date) != 0:
            date = date[0]
        if len(date) == DATE_LENGTH:
            fed_speech_transcript_links.append(link)
    return fed_speech_transcript_links

In [159]:
fed_page_links = get_fed_speech_links(doc_body)
fed_page_transcript_links = get_fed_speech_transcript_links(fed_page_links)

In [161]:
def filter_duplicate_links(speech_links):
    uniq_links = set(speech_links)
    return list(uniq_links)

In [162]:
speech_links = filter_duplicate_links(fed_page_transcript_links)

In [168]:
def get_fed_speeches(fed_speech_links):
    fed_speeches = []
    for link in fed_speech_links:
        fed_speeches.append(requests.get(FED_BASE_URL + link).text)
    return fed_speeches

In [169]:
fed_speeches = get_fed_speeches(speech_links)

In [171]:
print(len(fed_speeches))

20
